In [5]:
import re

def parse_fact(fact):
    """Parse 'Name(predicate)' → ('predicate', 'Name')"""
    m = re.match(r'(\w+)\((\w+)\)', fact)
    if not m:
        raise ValueError(f"Invalid fact format: {fact}")
    entity, pred = m.group(1), m.group(2)
    return (pred, entity)

def fact_to_str(fact):
    return f"{fact[0]}({fact[1]})"

def match_fact(rule_fact, known_fact):
    rule_pred, rule_arg = rule_fact
    known_pred, known_arg = known_fact
    if rule_pred != known_pred:
        return None
    if rule_arg[0].isupper():
        return {rule_arg: known_arg}
    elif rule_arg == known_arg:
        return {}
    else:
        return None

def apply_bindings(conclusion, bindings):
    pred, arg = conclusion
    if arg in bindings:
        arg = bindings[arg]
    return (pred, arg)

def forward_chaining(facts, rules, goal=None):
    known_facts = set(facts)
    new_inferred = True

    print("Initial facts:")
    for f in known_facts:
        print("  ", fact_to_str(f))
    print("\n--- Forward Chaining Steps ---")

    while new_inferred:
        new_inferred = False
        for premises, conclusion in rules:
            for fact in list(known_facts):
                bindings = match_fact(premises[0], fact)
                if bindings is not None:
                    new_fact = apply_bindings(conclusion, bindings)
                    if new_fact not in known_facts:
                        known_facts.add(new_fact)
                        new_inferred = True
                        print(f"Derived {fact_to_str(new_fact)} using {fact_to_str(premises[0])} → {fact_to_str(conclusion)}")
                        if goal and new_fact == goal:
                            print(f"\n✅ Goal {fact_to_str(goal)} reached!")
                            print("Stopping inference.\n")
                            return known_facts

    print("\n❌ Goal not reached.") if goal else None
    print("\nFinal Facts:")
    for f in known_facts:
        print("  ", fact_to_str(f))
    return known_facts


# --- Example Knowledge Base ---

facts = [
    parse_fact("Akbar(ruler)"),
    parse_fact("Birbal(minister)")
]

rules = [
    ( [("ruler", "X")], ("king", "X") ),
    ( [("minister", "X")], ("advisor", "X") ),
    ( [("king", "X")], ("powerful", "X") )
]

goal = ("powerful", "Akbar")  # 🎯 our goal fact

# Run forward chaining
final_facts = forward_chaining(facts, rules, goal)

print("\nAs a set:")
print(final_facts)


Initial facts:
   minister(Birbal)
   ruler(Akbar)

--- Forward Chaining Steps ---
Derived king(Akbar) using ruler(X) → king(X)
Derived advisor(Birbal) using minister(X) → advisor(X)
Derived powerful(Akbar) using king(X) → powerful(X)

✅ Goal powerful(Akbar) reached!
Stopping inference.


As a set:
{('powerful', 'Akbar'), ('minister', 'Birbal'), ('advisor', 'Birbal'), ('ruler', 'Akbar'), ('king', 'Akbar')}
